Load images

In [ ]:
import glob

In [ ]:
fish = glob.glob("/root/data/alok/filtration_classifier_data/nonzero_crops/images_resized/*")
nofish = glob.glob("/root/data/alok/filtration_classifier_data/zero_crops/images_resized/*")
print("Fish {}, no fish {}".format(len(fish), len(nofish)))
good_images = glob.glob("/root/data/priority_queue/frames/good/*.jpg")
print("Number of good images {}".format(len(good_images)))
bad_images = glob.glob("/root/data/priority_queue/frames/bad/*/*.jpg")
print("Number of bad images {}".format(len(bad_images)))

Load model

In [ ]:
!nvidia-smi

In [ ]:
import keras

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [ ]:
model = keras.models.load_model("/root/data/priority_queue/models/draft_5.h5")

Generator

In [ ]:
import random
import numpy as np
import cv2

In [ ]:
import json
image_classes = json.load(open("/root/data/priority_queue/frames/image_classes.json"))

In [ ]:
class ForwardDataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, images, datatype = "bad", batch_size=16, dim=(224, 224, 3)):
        'Initialization'
        self.images = images
        self.batch_size = batch_size
        self.dim = dim
        self.datatype = datatype
        self.on_epoch_end()
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(len(self.images) / self.batch_size)
    
    def __getitem__(self, index):
        'Generate one batch of data'
        xbatch = []
        ybatch = []
        start = index*self.batch_size
        end = (index+1)*self.batch_size
        for i in range(start, end, 1):
            path = self.images[i]
            image_name= os.path.basename(path)
            image = cv2.imread(path)
            image = cv2.resize(image, (self.dim[0], self.dim[1]))
            xbatch.append(image)
            if self.datatype == "good":
                ybatch.append([1, 0, 0])
            elif self.datatype == "bad":
                ybatch.append(image_classes.get(image_name, None))

        return np.array(xbatch), np.array(ybatch)
    
    def on_epoch_end(self):
        # random.shuffle(self.bad_images)
        return

In [ ]:
gen = ForwardDataGenerator(good_images)

Prediction

In [ ]:
xb, yb = gen[0]

In [ ]:
ypred = model.predict_on_batch(xb)
ypred

Forward pass on all images

In [ ]:
from tqdm import tqdm

In [ ]:
gen = ForwardDataGenerator(bad_images)

In [ ]:
all_preds = []
for (xb, yb) in tqdm(gen):
    ypred = model.predict_on_batch(xb)
    all_preds.append(ypred)

In [ ]:
from time import time

In [ ]:
start = time()
# model.predict_on_batch(np.expand_dims(xb[0, ...], axis=0))
image = cv2.imread(good_images[0])
image = cv2.resize(image, (224, 224))
model.predict_on_batch(np.expand_dims(image, axis=0))
# ypred = model.predict_on_batch(xb)
end = time()
print(end-start)

Compare

In [ ]:
# batch_acc = np.sum(yb == np.round(ypred).squeeze()) / 16.0
# print("Batch accuracy {}".format(batch_acc))

Let's experiment

In [ ]:
# take a good image
path = np.random.choice(good_images)
image = cv2.imread(path)
image = cv2.resize(image, (224, 224))
pred = model.predict_on_batch(np.expand_dims(image, axis=0))[0]
print("prediction {}".format(pred))

In [ ]:
from imgaug import augmenters as iaa
import matplotlib.pyplot as plt

In [ ]:
aug = iaa.GaussianBlur((0, 3.0), name="GaussianBlur")

In [ ]:
augmented = aug.augment_image(image)

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(20, 10))
ax[0].imshow(image)
ax[1].imshow(augmented)
plt.show()

Blur experiment

In [ ]:
predictions = []
_range = np.arange(0.0, 3.0, 0.05)
for i in _range:
    aug = iaa.GaussianBlur((i, i), name="GaussianBlur")
    augmented = aug.augment_image(image)
#     plt.imshow(augmented)
#     plt.show()
    pred = model.predict_on_batch(np.expand_dims(augmented, axis=0))[0]
#     print("prediction {}".format(pred))
    predictions.append(pred)

In [ ]:
predictions = np.array(predictions)

In [ ]:
predictions.shape

In [ ]:
plt.plot(_range, predictions[:, 0])
plt.plot(_range, predictions[:, 1])
plt.plot(_range, predictions[:, 2])
plt.plot(_range, predictions[:, 3])
plt.plot(_range, predictions[:, 4])
plt.xlabel("Blur amount")
plt.ylabel("Probability of being a good image")
plt.legend(["good", "blurry", "dark", "obstructed", "bad"])
plt.show()

Darkness experiment

In [ ]:
predictions = []
_range = np.arange(0, -100, -2)
for i in _range:
    aug = iaa.Add((i, i))
    augmented = aug.augment_image(image)
#     plt.imshow(augmented)
#     plt.show()
    pred = model.predict_on_batch(np.expand_dims(augmented, axis=0))[0]
#     print("prediction {}".format(pred))
    predictions.append(pred)

In [ ]:
predictions = np.array(predictions)

In [ ]:
plt.plot(_range, predictions[:, 0])
plt.plot(_range, predictions[:, 1])
plt.plot(_range, predictions[:, 2])
plt.plot(_range, predictions[:, 3])
plt.plot(_range, predictions[:, 4])
plt.xlabel("Darkness amount")
plt.ylabel("Probability of being a good image")
plt.legend(["good", "blurry", "dark", "obstructed", "bad"])
plt.xlim([_range[0], _range[-1]])
plt.show()

Obstruction experiment

In [ ]:
from tqdm import tqdm

In [ ]:
predictions = []
_range = np.arange(0, 10, 1)
for i in tqdm(_range):
    aug = iaa.CoarseDropout(size_percent=i/100, p=0.2)
    augmented = aug.augment_image(image)
#     plt.imshow(augmented)
#     plt.show()
    pred = model.predict_on_batch(np.expand_dims(augmented, axis=0))[0]
#     print("prediction {}".format(pred))
    predictions.append(pred)
predictions = np.array(predictions)

In [ ]:
aug = iaa.CoarseDropout(size_percent=10/100, p=0.3)
augmented = aug.augment_image(image)
plt.imshow(augmented)
plt.show()

In [ ]:
plt.plot(_range, predictions[:, 0])
plt.plot(_range, predictions[:, 1])
plt.plot(_range, predictions[:, 2])
plt.plot(_range, predictions[:, 3])
plt.plot(_range, predictions[:, 4])
plt.xlabel("Obstruction amount")
plt.ylabel("Probability of being a good image")
plt.legend(["good", "blurry", "dark", "obstructed", "bad"])
plt.xlim([_range[-1], _range[0]])
plt.show()